In [1]:
!apt install python3.10-venv
!python -m venv audio_env
!source activate audio_env/bin/activate

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-pip-whl python3-setuptools-whl
The following NEW packages will be installed:
  python3-pip-whl python3-setuptools-whl python3.10-venv
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 2,473 kB of archives.
After this operation, 2,882 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip-whl all 22.0.2+dfsg-1ubuntu0.3 [1,679 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-setuptools-whl all 59.6.0-1.2ubuntu0.22.04.1 [788 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3.10-venv amd64 3.10.12-1~22.04.2 [5,724 B]
Fetched 2,473 kB in 2s (1,089 kB/s)
Selecting previously unselected package python3-pip-whl.
(Reading database ... 120511 files and directories currently installed.)
Pr

In [2]:
!pip install datasets
!pip install wandb
!pip install transformers==4.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=4510bcea704e69473b18614b0bab5cf9ef2e4e94d88481e236a8304daa4a9b28
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8

In [3]:
import os
import logging
import librosa

import wandb
import numpy as np

from datasets import DatasetDict, load_dataset, load_metric
from transformers import (
    HubertForSequenceClassification,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    Wav2Vec2FeatureExtractor,
    HubertConfig,
)

In [26]:
model = "facebook/hubert-base-ls960"
#model = "lighthubert/lighthubert_small.pt"
feature_extractor = Wav2Vec2FeatureExtractor(
          sampling_rate=8000,
          truncation=True
)
print(feature_extractor)

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 8000,
  "truncation": true
}



In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# model_id = "facebook/hubert-base-ls960"
# feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_id)
ds_path = "/content/drive/MyDrive/Data/Diss/train/cv-valid-train/"
audio_file = ds_path + "0/sample-000030.mp3"
audio_array = librosa.load(audio_file, sr=8000, mono=False)[0]
input = feature_extractor(
       raw_speech=audio_array,
       sampling_rate=8000,
       padding=True,
      return_tensors="pt"
)
print(input)
#print(librosa.get_samplerate(audio_file))
print(input.input_values.shape)
print(audio_array.shape)

{'input_values': tensor([[-0.0011, -0.0011, -0.0011,  ..., -0.0009, -0.0008, -0.0010]])}
torch.Size([1, 41280])
(41280,)


In [29]:
audio_file_1 = ds_path + "0/sample-000031.mp3"
audio_file_2 = ds_path + "0/sample-000032.mp3"
audio_array_1 = librosa.load(audio_file_1, sr=8000, mono=False)[0]
audio_array_2 = librosa.load(audio_file_2, sr=8000, mono=False)[0]
input_with_one_audio = feature_extractor(
       audio_array_1,
       sampling_rate=8000,
       padding=True,
      return_tensors="pt"
)
input_with_two_audios = feature_extractor(
       [audio_array_1, audio_array_2],
       sampling_rate=8000,
       padding=True,
      return_tensors="pt"
)
print(input_with_one_audio.input_values.shape)
print(input_with_two_audios.input_values.shape)

torch.Size([1, 61824])
torch.Size([2, 71424])


In [30]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # visualizing data
import seaborn as sns # visualizing data with stunning default theme
import sklearn # contain algorithms
import warnings
warnings.filterwarnings('ignore')

# load dataset from input directory
df = pd.read_csv("/content/drive/MyDrive/Data/Diss/cv-valid-train.csv")
df[df['age'].notna()].head()
df =  df.head(int(len(df)*0.005))
print(len(df))
df.tail()

978


,filename,text,up_votes,down_votes,age,gender,accent,duration
973,cv-valid-train/sample-000973.mp3,he didn't want any conversation at this point,1,0,twenties,male,indian,NaN
974,cv-valid-train/sample-000974.mp3,they were all furious when the escapade failed,1,0,teens,male,us,NaN
975,cv-valid-train/sample-000975.mp3,it always helps to be learning something new a...,1,0,NaN,NaN,NaN,NaN
976,cv-valid-train/sample-000976.mp3,where are you bound asked the young arab,1,0,thirties,female,us,NaN
977,cv-valid-train/sample-000977.mp3,at that time it was quite clear in my own mind...,2,0,sixties,male,england,NaN


In [31]:
#we extract the columns that we think useful are
df = df[['filename','age','gender']]
#To clean the data we remove the sample with NaN attribute values.
data = df[df['age'].notna() & df['gender'].notna()]
data.reset_index(inplace=True, drop=True)
data.head()

,filename,age,gender
0,cv-valid-train/sample-000005.mp3,twenties,female
1,cv-valid-train/sample-000008.mp3,seventies,male
2,cv-valid-train/sample-000013.mp3,thirties,female
3,cv-valid-train/sample-000014.mp3,sixties,male
4,cv-valid-train/sample-000019.mp3,fifties,male


In [32]:
#if the below code fail to covert gender datatype and values then uncomment above code
cleanup_nums = {"gender": {"male":1,"female":0,"other":0.5}}
data = data.replace(cleanup_nums)
data.head()

,filename,age,gender
0,cv-valid-train/sample-000005.mp3,twenties,0.0
1,cv-valid-train/sample-000008.mp3,seventies,1.0
2,cv-valid-train/sample-000013.mp3,thirties,0.0
3,cv-valid-train/sample-000014.mp3,sixties,1.0
4,cv-valid-train/sample-000019.mp3,fifties,1.0


In [33]:
 print(len(data['age'].unique()) )

7


In [34]:
NUM_LABELS = len(data['age'].unique())
model_id = "facebook/hubert-base-ls960"

config = HubertConfig.from_pretrained(model_id, num_labels=NUM_LABELS)
hubert_model = HubertForSequenceClassification.from_pretrained(
    model_id,
    config=config,  # because we need to update num_labels as per our dataset
    ignore_mismatched_sizes=True,  # to avoid classifier size mismatch from from_pretrained.
)

loading configuration file https://huggingface.co/facebook/hubert-base-ls960/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/675e5285912f93e32d794e5b9ef2cb071172e75ff2069404390dbd17d3c57744.5e8693dc119c292873875f9ab86e53c4144812404bb8c44c14d6ab91b6d63b8d
Model config HubertConfig {
  "_name_or_path": "facebook/hubert-base-ls960",
  "activation_dropout": 0.1,
  "apply_spec_augment": true,
  "architectures": [
    "HubertModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": 

In [35]:
# freeze all layers to begin with
for param in hubert_model.parameters():
    param.requires_grad = False

# freeze two encoder layers
layers_freeze_num = 2
n_layers = (
    4 + layers_freeze_num * 16
)  # 4 refers to projector and classifier's weights and biases.
for name, param in list(hubert_model.named_parameters())[-n_layers:]:
    param.requires_grad = True

In [36]:
PROJECT_ROOT = "/content/drive/MyDrive/Data/Diss/"
dataset_config = {
  "LOADING_SCRIPT_FILES": os.path.join(PROJECT_ROOT, "common_voice.py"),
  "CONFIG_NAME": "clean",
  "DATA_DIR": os.path.join(PROJECT_ROOT, "trainn"),
  "CACHE_DIR": os.path.join(PROJECT_ROOT, "cache_crema"),
}
ds = load_dataset(
  dataset_config["LOADING_SCRIPT_FILES"],
  dataset_config["CONFIG_NAME"],
  data_dir=dataset_config["DATA_DIR"],
  cache_dir=dataset_config["CACHE_DIR"]
)
print(ds)

DatasetDict({
    train: Dataset({
        features: ['file', 'label'],
        num_rows: 7310
    })
})


In [37]:
# CONVERING RAW AUDIO TO ARRAYS
ds = ds.map(
        lambda x: {
            "array": librosa.load(x["file"], sr=8000, mono=False)[0]
        },
        num_proc=8,
    )

In [38]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['file', 'label', 'array'],
        num_rows: 7310
    })
})


In [39]:
# LABEL TO ID
ds = ds.class_encode_column("label")

In [40]:
# PROCESS THE DATASET TO THE FORMAT EXPECTED BY THE MODEL FOR TRAINING

INPUT_FIELD = "input_values"
LABEL_FIELD = "labels"

def prepare_dataset(batch, feature_extractor):
    audio_arr = batch["array"]
    input = feature_extractor(
        audio_arr, sampling_rate=8000, padding=True, return_tensors="pt"
    )

    batch[INPUT_FIELD] = input.input_values[0]
    batch[LABEL_FIELD] = batch[
        "label"
    ]  # colname MUST be labels as Trainer will look for it by default

    return batch

In [41]:
# APPLY THE DATA PREP USING FEATURE EXTRACTOR TO ALL EXAMPLES
ds = ds.map(
    prepare_dataset,
    fn_kwargs={"feature_extractor": feature_extractor},
    num_proc=2,
)
logging.info("Finished extracting features from audio arrays.")

In [42]:
# INTRODUCE TRAIN TEST VAL SPLITS

# 90% train, 10% test + validation
train_testvalid = ds["train"].train_test_split(shuffle=True, test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
ds = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'val': test_valid['train']})

In [43]:
trainer_config = {
  "OUTPUT_DIR": "results",
  "TRAIN_EPOCHS": 3,
  "TRAIN_BATCH_SIZE": 1,
  "EVAL_BATCH_SIZE": 1,
  "GRADIENT_ACCUMULATION_STEPS": 4,
  "WARMUP_STEPS": 500,
  "DECAY": 0.01,
  "LOGGING_STEPS": 10,
  "MODEL_DIR": "models/test-hubert-model",
  "SAVE_STEPS": 100
}

# Fine-Tuning with Trainer
training_args = TrainingArguments(
    output_dir=trainer_config["OUTPUT_DIR"],  # output directory
    gradient_accumulation_steps=trainer_config[
        "GRADIENT_ACCUMULATION_STEPS"
    ],  # accumulate the gradients before running optimization step
    num_train_epochs=trainer_config[
        "TRAIN_EPOCHS"
    ],  # total number of training epochs
    per_device_train_batch_size=trainer_config[
        "TRAIN_BATCH_SIZE"
    ],  # batch size per device during training
    per_device_eval_batch_size=trainer_config[
        "EVAL_BATCH_SIZE"
    ],  # batch size for evaluation
    warmup_steps=trainer_config[
        "WARMUP_STEPS"
    ],  # number of warmup steps for learning rate scheduler
    save_steps=trainer_config["SAVE_STEPS"], # save checkpoint every 100 steps
    weight_decay=trainer_config["DECAY"],  # strength of weight decay
    logging_steps=trainer_config["LOGGING_STEPS"],
    evaluation_strategy="epoch",  # report metric at end of each epoch
    report_to="none",  # enable logging to W&B
)

PyTorch: setting up devices


In [44]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union

import torch
from transformers import Wav2Vec2Processor

INPUT_FIELD = "input_values"
LABEL_FIELD = "labels"


@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(
        self, examples: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:

        input_features = [
            {INPUT_FIELD: example[INPUT_FIELD]} for example in examples
        ]  # example is basically row0, row1, etc...
        labels = [example[LABEL_FIELD] for example in examples]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        batch[LABEL_FIELD] = torch.tensor(labels)

        return batch

In [46]:
# DEFINE DATA COLLATOR - TO PAD TRAINING BATCHES DYNAMICALLY
data_collator = DataCollatorCTCWithPadding(
    processor=feature_extractor, padding=True
)


def compute_metrics(eval_pred):
    # DEFINE EVALUATION METRIC
    compute_accuracy_metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return compute_accuracy_metric.compute(predictions=predictions, references=labels)

# START TRAINING
trainer = Trainer(
    model=hubert_model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=ds["train"],  # training dataset
    eval_dataset=ds["val"],  # evaluation dataset
    compute_metrics=compute_metrics,
)

RuntimeError: ignored

In [ ]:
trainer.train()

In [ ]:
test_results = trainer.predict(ds["test"])